In [1]:
# Handle to the workspace
from azure.ai.ml import MLClient

# Authentication package
from azure.identity import DefaultAzureCredential

credential = DefaultAzureCredential()

In [2]:
# Parameters (make sure these are correct)
subscr_id = "" 
resc_name = ""
worksp_name = ""

# Get a handle to the workspace
ml_client = MLClient(
    credential = credential,
    subscription_id = subscr_id,
    resource_group_name = resc_name,
    workspace_name = worksp_name,
)

# ws 
# from azureml.core import Workspace
# ws = Workspace(subscr_id, resc_name, worksp_name)

In [ ]:
### First run az login in a command prompt to make sure you have authenticated using the Azure CLI

from azure.ai.ml.entities import AmlCompute

cpu_compute_target = ""
cpu_size = ""

gpu_compute_target = ""
gpu_size = ""

def create_compute(target, size):
    try:
        # let's see if the compute target already exists
        cluster = ml_client.compute.get(target)
        print(
            f"You already have a cluster named {target}, we'll reuse it as is."
        )

    except Exception:
        print("Creating a new compute target...")

        # Let's create the Azure ML compute object with the intended parameters
        cluster = AmlCompute(
            # Name assigned to the compute cluster
            name=target,
            # Azure ML Compute is the on-demand VM service
            type="amlcompute",
            # VM Family
            size=size,
            # Minimum running nodes when there is no job running
            min_instances=0,
            # Nodes in cluster
            max_instances=4,
            # How many seconds will the node running after the job termination
            idle_time_before_scale_down=180,
            # Dedicated or LowPriority. The latter is cheaper but there is a chance of job termination
            tier="Dedicated",
        )

        # Now, we pass the object to MLClient's create_or_update method
        cluster = ml_client.begin_create_or_update(cluster).result()

    print(
        f"AMLCompute with name {cluster.name} is created, the compute size is {cluster.size}"
    )
    
    return target
    
compute_target = create_compute(gpu_compute_target, gpu_size)

In [4]:
# For job
env_name = "detectron2-env@latest"

In [5]:
from azure.ai.ml import MLClient, command, Input
from azure.ai.ml import command, Input, MLClient, UserIdentityConfiguration, ManagedIdentityConfiguration
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes, InputOutputModes
from azure.identity import DefaultAzureCredential
import os

# Load from local
data_path = r""

# Load from Azure container
data_path = r""

# If Azure container, you may need identity
identity = UserIdentityConfiguration() # Use the user's identity
# identity = ManagedIdentityConfiguration() # Use the compute target managed identity


In [ ]:
data_path

In [14]:
from azure.ai.ml import command
from azure.ai.ml import Input

# Set job and run
job = command(
     code=".",  
     command='python detectron2_train.py --traindata ${{inputs.train_data}} --epochs ${{inputs.epochs}} --learning_rate ${{inputs.learning_rate}} --img_per_batch ${{inputs.img_per_batch}} --roi_batch_size ${{inputs.roi_batch_size}} --roi_batch_size ${{inputs.rescale}}' ,
     inputs={
         "train_data": Input(path=data_path,
             type=AssetTypes.URI_FOLDER,
             mode=InputOutputModes.RO_MOUNT # Mount (.RO_MOUNT) or download (.DOWNLOAD)
         ),
         "epochs": 10, # iterations / epoch = num_train_images / img_per_batch
         "learning_rate": 0.0025,
         "gamma": 0.01, # reduce learning rate at 2/5, 3/5 and 4/5 of the iterations
         "img_per_batch": 4, # the real "batch size"
         "roi_batch_size": 16, # the ROI head "batch size"
         "rescale": 224,
     },
     compute=compute_target,
     environment=env_name,
     identity=identity,
     outputs={},  # Add any required outputs here
     description="Detectron2 training."
 )

# ml_client.jobs.create_or_update(job)

In [15]:
from azure.ai.ml.sweep import Choice, Uniform, MedianStoppingPolicy

num_epochs = 1 # Iterations over entire training dataset

job_for_sweep = job(
    epochs=Choice(values=[num_epochs]),
    learning_rate=Uniform(min_value=0.0001, max_value=0.01),
    gamma=Uniform(min_value=0.01, max_value=0.1),
    img_per_batch=Choice(values=[2,3,5]),
    roi_batch_size=Choice(values=[4, 8, 16, 64, 128]),
    rescale=Choice(values=[538])
)

In [ ]:
# apply the sweep parameter to obtain the sweep_job
sweep_job = job_for_sweep.sweep(
    compute=compute_target,
    sampling_algorithm="random",
    primary_metric="epoch_loss",
    goal="Minimize",
)

# define the limits for this sweep
sweep_job.set_limits(max_total_trials=20, max_concurrent_trials=1)

# define stopping policy
# sweep_job.early_termination = MedianStoppingPolicy(delay_evaluation = int(num_epochs/4), evaluation_interval = int(num_epochs/4))

# submit the sweep
ml_client.create_or_update(sweep_job)
